# Lab 1.4.5 Solutions: Probability Distributions Lab

This notebook contains solutions for the exercises in the Probability Distributions lab.

---

> **📝 Note:** This solution notebook is designed to be self-contained and includes all necessary
> helper functions. However, if you prefer to use your own implementations from the main notebook,
> ensure those cells have been run first.
>
> You can also import production-ready implementations from the scripts:
> ```python
> from scripts.math_utils import sigmoid, relu, Adam, SGD
> from scripts.visualization_utils import plot_loss_landscape, plot_training_curve
> ```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

print("Probability Distributions Solutions")
print("=" * 50)

## Helper Functions from the Main Notebook

In [ ]:
def bernoulli_pmf(k, p):
    """Bernoulli probability mass function: p(k) = p^k * (1-p)^(1-k)"""
    return (p ** k) * ((1 - p) ** (1 - k))

def bernoulli_log_pmf(k, p):
    """Log of Bernoulli PMF"""
    eps = 1e-10
    p = np.clip(p, eps, 1 - eps)
    return k * np.log(p) + (1 - k) * np.log(1 - p)

def softmax(logits):
    """Convert logits to probabilities"""
    exp_logits = np.exp(logits - np.max(logits))
    return exp_logits / exp_logits.sum()

print("Helper functions defined!")

---

## Exercise Solution: Implement Binary Cross-Entropy

### 🧒 ELI5: What is Binary Cross-Entropy?

> **Imagine you're judging a weather forecaster...**
>
> - Forecaster says: "70% chance of rain" (p = 0.7)
> - Reality: It rained! (y = 1)
>
> How do we score them?
> - If they said 90% rain and it rained: Small penalty (good prediction!)
> - If they said 10% rain and it rained: Large penalty (bad prediction!)
>
> **BCE formula:** `loss = -[y*log(p) + (1-y)*log(1-p)]`
>
> This is EXACTLY the same as asking:
> "What's the negative log-probability of the outcome under a Bernoulli distribution?"

### The Math

**Bernoulli log-PMF:**
$$\log p(y|\hat{p}) = y \log(\hat{p}) + (1-y) \log(1-\hat{p})$$

**Binary Cross-Entropy (negative log-PMF):**
$$BCE = -[y \log(\hat{p}) + (1-y) \log(1-\hat{p})]$$

They're the same thing, just with opposite signs!

In [ ]:
def binary_cross_entropy(y_true, y_pred):
    """
    Binary Cross-Entropy Loss.
    
    Formula: BCE = -[y*log(p) + (1-y)*log(1-p)]
    
    This is the standard loss for binary classification.
    
    Args:
        y_true: Binary labels (0 or 1)
        y_pred: Predicted probabilities (between 0 and 1)
        
    Returns:
        Mean BCE loss over all samples
    """
    # Clip predictions to avoid log(0) = -inf
    eps = 1e-10
    y_pred = np.clip(y_pred, eps, 1 - eps)
    
    # Compute BCE for each sample
    bce = -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    
    # Return mean
    return np.mean(bce)

print("binary_cross_entropy() function defined!")

In [ ]:
# Test data
y_true_binary = np.array([1, 0, 1, 1, 0])
y_pred_probs = np.array([0.9, 0.2, 0.8, 0.7, 0.3])

print("Test Data:")
print(f"  True labels:  {y_true_binary}")
print(f"  Predictions:  {y_pred_probs}")

In [ ]:
# Compute BCE using our function
bce = binary_cross_entropy(y_true_binary, y_pred_probs)

# Compute as Bernoulli NLL (negative log-PMF)
bernoulli_nll_values = [-bernoulli_log_pmf(y, p) for y, p in zip(y_true_binary, y_pred_probs)]
bernoulli_nll_mean = np.mean(bernoulli_nll_values)

print("\nComparison: BCE vs Bernoulli NLL")
print("=" * 50)
print(f"Binary Cross-Entropy: {bce:.6f}")
print(f"Bernoulli NLL (mean): {bernoulli_nll_mean:.6f}")
print(f"\nDifference: {abs(bce - bernoulli_nll_mean):.10f}")
print("\n✅ They're the same! BCE = Bernoulli NLL")

In [ ]:
# Show per-sample breakdown
print("\nPer-Sample Breakdown:")
print("=" * 60)
print(f"{'Sample':<8} {'y':<5} {'p̂':<8} {'BCE':<12} {'Bernoulli NLL':<12}")
print("-" * 60)

for i, (y, p) in enumerate(zip(y_true_binary, y_pred_probs)):
    bce_i = -(y * np.log(p + 1e-10) + (1-y) * np.log(1-p + 1e-10))
    nll_i = -bernoulli_log_pmf(y, p)
    print(f"{i:<8} {y:<5} {p:<8.2f} {bce_i:<12.6f} {nll_i:<12.6f}")

print("-" * 60)
print(f"{'Mean:':<14} {'':<8} {bce:<12.6f} {bernoulli_nll_mean:<12.6f}")

### Why Does This Matter?

Understanding that **BCE = Bernoulli NLL** tells us:

1. **Loss functions aren't arbitrary** - they come from probability theory
2. **Minimizing BCE** = Finding parameters that maximize the likelihood of observing our data
3. **Model outputs are probabilities** - the sigmoid/softmax isn't just for "squashing", it's giving us actual probability distributions

---

## Bonus: Visualizing BCE Behavior

In [ ]:
# Visualize how BCE penalizes predictions

p_range = np.linspace(0.01, 0.99, 100)

# BCE when y = 1 (should predict high)
bce_y1 = -np.log(p_range)

# BCE when y = 0 (should predict low)
bce_y0 = -np.log(1 - p_range)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot for each true label
axes[0].plot(p_range, bce_y1, 'b-', linewidth=2, label='True label y=1')
axes[0].plot(p_range, bce_y0, 'r-', linewidth=2, label='True label y=0')
axes[0].set_xlabel('Predicted probability p̂', fontsize=12)
axes[0].set_ylabel('BCE Loss', fontsize=12)
axes[0].set_title('BCE Loss vs Predicted Probability', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(0, 5)

# Gradient (derivative)
grad_y1 = -1 / p_range  # d/dp [-log(p)] = -1/p
grad_y0 = 1 / (1 - p_range)  # d/dp [-log(1-p)] = 1/(1-p)

axes[1].plot(p_range, np.abs(grad_y1), 'b-', linewidth=2, label='True label y=1')
axes[1].plot(p_range, np.abs(grad_y0), 'r-', linewidth=2, label='True label y=0')
axes[1].set_xlabel('Predicted probability p̂', fontsize=12)
axes[1].set_ylabel('|Gradient|', fontsize=12)
axes[1].set_title('Gradient Magnitude vs Predicted Probability', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(0, 20)

plt.tight_layout()
plt.show()

print("📊 Key Observations:")
print("  - When y=1: Loss decreases as p̂ → 1 (predicting positive correctly)")
print("  - When y=0: Loss decreases as p̂ → 0 (predicting negative correctly)")
print("  - Gradients are LARGER for worse predictions (faster learning when very wrong!)")

---

## Bonus 2: Comparing BCE with MSE for Classification

In [ ]:
# Why don't we use MSE for classification?

def mse_loss(y_true, y_pred):
    """Mean Squared Error"""
    return np.mean((y_true - y_pred) ** 2)

# For y = 1, compare losses as a function of prediction
p_range = np.linspace(0.01, 0.99, 100)

bce_values = -np.log(p_range)  # BCE when y=1
mse_values = (1 - p_range) ** 2  # MSE when y=1

# Gradients
bce_grad = 1 / p_range  # |gradient| of BCE
mse_grad = 2 * (1 - p_range)  # |gradient| of MSE

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss comparison
axes[0].plot(p_range, bce_values, 'b-', linewidth=2, label='BCE')
axes[0].plot(p_range, mse_values, 'r-', linewidth=2, label='MSE')
axes[0].set_xlabel('Predicted probability p̂ (true y=1)', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('BCE vs MSE Loss (when y=1)', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(0, 5)

# Gradient comparison
axes[1].plot(p_range, bce_grad, 'b-', linewidth=2, label='BCE gradient')
axes[1].plot(p_range, mse_grad, 'r-', linewidth=2, label='MSE gradient')
axes[1].set_xlabel('Predicted probability p̂ (true y=1)', fontsize=12)
axes[1].set_ylabel('|Gradient|', fontsize=12)
axes[1].set_title('BCE vs MSE Gradient (when y=1)', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(0, 10)

plt.tight_layout()
plt.show()

print("📊 Why BCE is better for classification:")
print("  - MSE gradient → 0 when p̂ → 0 (model is VERY wrong but learns slowly!)")
print("  - BCE gradient → ∞ when p̂ → 0 (model learns FAST when very wrong)")
print("  - This is called the 'vanishing gradient problem' with MSE for classification")

---

## Bonus 3: Implementing Multi-class Cross-Entropy

In [ ]:
def cross_entropy(y_true_onehot, y_pred_probs):
    """
    Multi-class Cross-Entropy Loss.
    
    Formula: CE = -sum(y_k * log(p_k))
    
    This is the standard loss for multi-class classification.
    
    Args:
        y_true_onehot: One-hot encoded true labels (shape: [n_samples, n_classes])
        y_pred_probs: Predicted probabilities (shape: [n_samples, n_classes])
        
    Returns:
        Mean CE loss over all samples
    """
    eps = 1e-10
    y_pred_probs = np.clip(y_pred_probs, eps, 1 - eps)
    
    # For one-hot labels, this simplifies to -log(p_true_class)
    ce = -np.sum(y_true_onehot * np.log(y_pred_probs), axis=-1)
    
    return np.mean(ce)

# Test data: 3 samples, 5 classes
n_samples = 3
n_classes = 5

# True labels (as indices)
true_classes = np.array([0, 2, 4])

# Convert to one-hot
y_true_onehot = np.eye(n_classes)[true_classes]

# Predicted logits (network output)
logits = np.array([
    [2.0, 0.5, 0.1, -0.5, -1.0],  # Should predict class 0
    [0.1, 0.3, 2.5, 0.2, -0.1],   # Should predict class 2
    [-0.5, -0.3, 0.1, 0.2, 1.8],  # Should predict class 4
])

# Convert to probabilities
y_pred_probs_multi = np.array([softmax(l) for l in logits])

# Compute cross-entropy
ce_loss = cross_entropy(y_true_onehot, y_pred_probs_multi)

print("Multi-class Cross-Entropy Example")
print("=" * 50)
print(f"True classes: {true_classes}")
print(f"\nPredicted probabilities:")
for i, probs in enumerate(y_pred_probs_multi):
    pred_class = np.argmax(probs)
    print(f"  Sample {i}: {probs.round(3)} → pred: {pred_class}, true: {true_classes[i]}")
print(f"\nCross-Entropy Loss: {ce_loss:.6f}")

In [ ]:
# Verify: CE should equal -log(p_true_class) averaged over samples

manual_ce = []
for i, true_class in enumerate(true_classes):
    p_true = y_pred_probs_multi[i, true_class]
    nll = -np.log(p_true)
    manual_ce.append(nll)
    print(f"Sample {i}: p(true class {true_class}) = {p_true:.4f}, NLL = {nll:.4f}")

print(f"\nMean NLL: {np.mean(manual_ce):.6f}")
print(f"CE Loss:  {ce_loss:.6f}")
print(f"\n✅ They match! CE = mean of negative log-likelihood of true classes")

---

## Bonus 4: KL Divergence Connection

In [ ]:
def entropy(p):
    """Compute entropy of distribution p"""
    eps = 1e-10
    p = np.clip(p, eps, 1-eps)
    return -np.sum(p * np.log(p))

def kl_divergence(p, q):
    """Compute KL divergence D_KL(p || q)"""
    eps = 1e-10
    p = np.clip(p, eps, 1-eps)
    q = np.clip(q, eps, 1-eps)
    return np.sum(p * np.log(p / q))

def cross_entropy_probs(p, q):
    """Compute cross-entropy H(p, q)"""
    eps = 1e-10
    q = np.clip(q, eps, 1-eps)
    return -np.sum(p * np.log(q))

# Demonstration: CE = Entropy + KL Divergence
p_true = np.array([0.6, 0.3, 0.1])  # True distribution
q_pred = np.array([0.4, 0.4, 0.2])  # Predicted distribution

H_p = entropy(p_true)
D_kl = kl_divergence(p_true, q_pred)
H_pq = cross_entropy_probs(p_true, q_pred)

print("The Fundamental Relationship")
print("=" * 50)
print(f"True distribution P:      {p_true}")
print(f"Predicted distribution Q: {q_pred}")
print()
print(f"Entropy H(P):         {H_p:.6f}")
print(f"KL Divergence D(P||Q): {D_kl:.6f}")
print(f"Cross-Entropy H(P,Q):  {H_pq:.6f}")
print()
print(f"H(P) + D_KL(P||Q) = {H_p + D_kl:.6f}")
print()
print("✅ Cross-Entropy = Entropy + KL Divergence")
print("\n📊 Since Entropy(P) is constant during training:")
print("   Minimizing Cross-Entropy = Minimizing KL Divergence!")

---

## Summary: The Probability Foundations of Loss Functions

| Task | Distribution | Loss Function | Formula |
|------|--------------|---------------|---------|
| Regression | Gaussian | MSE | $(y - \hat{y})^2$ |
| Binary Classification | Bernoulli | BCE | $-[y\log(p) + (1-y)\log(1-p)]$ |
| Multi-class Classification | Categorical | CE | $-\sum_k y_k \log(p_k)$ |

**Key insight:** All these loss functions are just **negative log-likelihoods** of different probability distributions!

Training a neural network = Finding parameters that **maximize the likelihood** of observing our training data.

In [ ]:
import gc
gc.collect()

print("\n✅ Solution notebook complete!")
print("\n🎉 Congratulations! You've completed all Module 1.4 solutions!")